In [1]:
project_id = 'elife-data-pipeline'
source_dataset = 'de_dev'
output_dataset = 'de_dev'
output_table_prefix = 'data_science_'

max_workers = 10
max_editors = 100
email = 'd.ecer@elifesciences.org'

In [2]:
import logging
from datetime import datetime, timedelta
from functools import partial
from concurrent.futures import ThreadPoolExecutor
from typing import List

import pandas as pd
from tqdm.auto import tqdm

import data_science_pipeline.configure_warnings  # pylint: disable=unused-import
import data_science_pipeline.configure_notebook_logging  # pylint: disable=unused-import

from data_science_pipeline.utils.europepmc import (
    EuropePMCApi,
    europepmc_requests_retry_session
)
from data_science_pipeline.utils.bq import to_gbq, is_bq_not_found_exception
from data_science_pipeline.utils.jupyter import (
    read_big_query as _read_big_query,
)

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [3]:
LOGGER = logging.getLogger(__name__)

logging.basicConfig(level='INFO')
logging.getLogger('data_science_pipeline').setLevel(logging.INFO)

In [4]:
editor_parsed_pubmed_links_table_name = '{output_dataset}.{prefix}{suffix}'.format(
    output_dataset=output_dataset,
    prefix=output_table_prefix,
    suffix='editor_pubmed_links'
)

editor_parsed_pubmed_ids_table_name = '{output_dataset}.{prefix}{suffix}'.format(
    output_dataset=output_dataset,
    prefix=output_table_prefix,
    suffix='editor_pubmed_ids'
)

In [5]:
read_big_query = partial(_read_big_query, project_id=project_id)

In [6]:
default_query_props = dict(project=project_id, dataset=source_dataset)

In [7]:
_sql = (
    '''
    SELECT
        person_id,
        pubmed_url,
        provenance.imported_timestamp as imported_timestamp
    FROM `{table}`
    ''').format(table=editor_parsed_pubmed_ids_table_name)

try:
    existing_editor_ids_and_pubmed_url_df = read_big_query(_sql)
except Exception as e:  # pylint: disable=broad-except
    if not is_bq_not_found_exception(e):
        raise
    print('table not found: %s' % editor_parsed_pubmed_ids_table_name)
    existing_editor_ids_and_pubmed_url_df = pd.DataFrame(columns=['person_id','pubmed_url', 'imported_timestamp'])
print("existing_editor_ids_and_pubmed_url_df length: ",len(existing_editor_ids_and_pubmed_url_df))
existing_editor_ids_and_pubmed_url_df.head(3)

> ```sql
> 
>     SELECT
>         person_id,
>         pubmed_url,
>         provenance.imported_timestamp as imported_timestamp
>     FROM `de_dev.data_science_editor_pubmed_ids`
>     
> ```

existing_editor_ids_and_pubmed_url_df length:  6506


/Users/hazal/elife_repo/data-science-dags/venv/lib/python3.7/site-packages/google/cloud/bigquery/table.py:1952: FutureWarning: Using .astype to convert from timezone-aware dtype to timezone-naive dtype is deprecated and will raise in a future version.  Use obj.tz_localize(None) or obj.tz_convert('UTC').tz_localize(None) instead
  df[column] = pandas.Series(df[column], dtype=dtypes[column])


,person_id,pubmed_url,imported_timestamp
0,15332,http://www.ncbi.nlm.nih.gov/pubmed/?term=Rutz%...,2021-09-14 20:12:26.199992+00:00
1,178962,https://www.ncbi.nlm.nih.gov/pubmed/?term=Nwao...,2021-09-14 20:12:26.199992+00:00
2,125237,https://pubmed.ncbi.nlm.nih.gov/?sort=date&siz...,2021-09-14 20:12:26.199992+00:00


In [10]:
existing_editor_ids_set = set(existing_editor_ids_and_pubmed_url_df['person_id'])
print("existing_editor_ids_set length :",len(existing_editor_ids_set))

existing_editor_ids_set length : 773


In [11]:
editor_parsed_pubmed_links_df = read_big_query(
    'SELECT * FROM `{table_name}`\nWHERE parsed_search_term IS NOT NULL'.format(
        table_name=editor_parsed_pubmed_links_table_name
    )
)
print("editor_parsed_pubmed_links_df length: ",len(editor_parsed_pubmed_links_df))
editor_parsed_pubmed_links_df.head(3)

> ```sql
> SELECT * FROM `de_dev.data_science_editor_pubmed_links`
> WHERE parsed_search_term IS NOT NULL
> ```

editor_parsed_pubmed_links_df length:  763


,is_ncbi_bibliography_url,name,parsed_search_term,person_id,pubmed_url,relevant_pubmed_ids,relevant_pubmed_urls,resolved_pubmed_url,search_term
0,False,Barnabas Daru,"{'exclude': None, 'include': {'author': ['Daru...",187442,https://pubmed.ncbi.nlm.nih.gov/?term=Daru+BH&...,"[32355257, 29083043, 30455213, 28919204]","[https://www.ncbi.nlm.nih.gov/pubmed/32355257,...",https://pubmed.ncbi.nlm.nih.gov/?term=Daru+BH&...,Daru BH
1,False,Yuuki Watanabe,"{'exclude': None, 'include': {'author': ['Wata...",126873,https://www.ncbi.nlm.nih.gov/pubmed/?term=Wata...,"[25902489, 23341596, 20946384]","[https://www.ncbi.nlm.nih.gov/pubmed/25902489,...",https://www.ncbi.nlm.nih.gov/pubmed/?term=Wata...,Watanabe YY[Author]
2,False,Safi Darden,"{'exclude': None, 'include': {'author': ['Dard...",43038,https://pubmed.ncbi.nlm.nih.gov/?size=200&term...,"[32900316, 32531279, 31748621, 30800389]","[https://www.ncbi.nlm.nih.gov/pubmed/32900316,...",https://pubmed.ncbi.nlm.nih.gov/?size=200&term...,Darden SK


In [12]:
merged_editor_parsed_pubmed_links_df = editor_parsed_pubmed_links_df.merge(
    existing_editor_ids_and_pubmed_url_df,
    how='left',
    on='person_id',
    suffixes=('', '_existing')
)
print("merged_editor_parsed_pubmed_links_df length: ",len(merged_editor_parsed_pubmed_links_df))
merged_editor_parsed_pubmed_links_df.head(3)

merged_editor_parsed_pubmed_links_df length:  6769


,is_ncbi_bibliography_url,name,parsed_search_term,person_id,pubmed_url,relevant_pubmed_ids,relevant_pubmed_urls,resolved_pubmed_url,search_term,pubmed_url_existing,imported_timestamp
0,False,Barnabas Daru,"{'exclude': None, 'include': {'author': ['Daru...",187442,https://pubmed.ncbi.nlm.nih.gov/?term=Daru+BH&...,"[32355257, 29083043, 30455213, 28919204]","[https://www.ncbi.nlm.nih.gov/pubmed/32355257,...",https://pubmed.ncbi.nlm.nih.gov/?term=Daru+BH&...,Daru BH,https://pubmed.ncbi.nlm.nih.gov/?term=Daru+BH&...,2021-09-14 20:12:26.199992+00:00
1,False,Barnabas Daru,"{'exclude': None, 'include': {'author': ['Daru...",187442,https://pubmed.ncbi.nlm.nih.gov/?term=Daru+BH&...,"[32355257, 29083043, 30455213, 28919204]","[https://www.ncbi.nlm.nih.gov/pubmed/32355257,...",https://pubmed.ncbi.nlm.nih.gov/?term=Daru+BH&...,Daru BH,https://pubmed.ncbi.nlm.nih.gov/?term=Daru+BH&...,2021-09-14 20:12:26.199992+00:00
2,False,Barnabas Daru,"{'exclude': None, 'include': {'author': ['Daru...",187442,https://pubmed.ncbi.nlm.nih.gov/?term=Daru+BH&...,"[32355257, 29083043, 30455213, 28919204]","[https://www.ncbi.nlm.nih.gov/pubmed/32355257,...",https://pubmed.ncbi.nlm.nih.gov/?term=Daru+BH&...,Daru BH,https://pubmed.ncbi.nlm.nih.gov/?term=Daru+BH&...,2021-09-15 07:29:43.577105+00:00


In [13]:
editors_with_changed_pubmed_url_df = merged_editor_parsed_pubmed_links_df[
    (merged_editor_parsed_pubmed_links_df['pubmed_url_existing'].notnull())
    &
    (merged_editor_parsed_pubmed_links_df['pubmed_url'] != merged_editor_parsed_pubmed_links_df['pubmed_url_existing'])
].drop(columns=['pubmed_url_existing','imported_timestamp'])

print("editors_with_changed_pubmed_url_df length: ",len(editors_with_changed_pubmed_url_df))
editors_with_changed_pubmed_url_df.head(3)

editors_with_changed_pubmed_url_df length:  284


,is_ncbi_bibliography_url,name,parsed_search_term,person_id,pubmed_url,relevant_pubmed_ids,relevant_pubmed_urls,resolved_pubmed_url,search_term
1681,False,Andrew King,"{'exclude': None, 'include': {'author': ['King...",14601,http://www.ncbi.nlm.nih.gov/pubmed?term=King%2...,"[31949136, 31300665, 29136122, 31802997, 28891...","[https://www.ncbi.nlm.nih.gov/pubmed/31949136,...",https://www.ncbi.nlm.nih.gov/pubmed?term=King%...,King A J[au]
4702,False,Yibing Shan,"{'exclude': None, 'include': {'author': ['Shan...",4398,http://www.ncbi.nlm.nih.gov/pubmed?term=Shan%2...,"[24918548, 23374350, 21545110, 20947758]","[https://www.ncbi.nlm.nih.gov/pubmed/24918548,...",https://www.ncbi.nlm.nih.gov/pubmed?term=Shan%...,Shan Y[au]
6448,False,Eduardo Franco,"{'exclude': {'pmid': ['22208813', '24331745', ...",1086,https://tinyurl.com/PubMedEduardoFranco,[],[],https://pubmed.ncbi.nlm.nih.gov/?term=franco+e...,franco el[Author] OR 30907564[pmid] OR 2644535...


In [14]:
new_added_editors_df = editor_parsed_pubmed_links_df[
    ~editor_parsed_pubmed_links_df['person_id'].isin(existing_editor_ids_set)
]

print("new_added_editors_df length: ",len(new_added_editors_df))
new_added_editors_df.head(3)

new_added_editors_df length:  1


,is_ncbi_bibliography_url,name,parsed_search_term,person_id,pubmed_url,relevant_pubmed_ids,relevant_pubmed_urls,resolved_pubmed_url,search_term
168,False,Lu Chen,"{'exclude': None, 'include': {'author': ['Chen...",90633,https://www.ncbi.nlm.nih.gov/pubmed/?term=Chen...,"[30782829, 30355624, 30068571, 29760176, 25843...","[https://pubmed.ncbi.nlm.nih.gov/30782829, htt...",https://www.ncbi.nlm.nih.gov/pubmed/?term=Chen...,Chen L[Author]


In [15]:
remaining_editor_parsed_pubmed_links_df = pd.concat([new_added_editors_df, editors_with_changed_pubmed_url_df])

print("remaining_editor_parsed_pubmed_links_df length: ",len(remaining_editor_parsed_pubmed_links_df))
remaining_editor_parsed_pubmed_links_df.head(3)

remaining_editor_parsed_pubmed_links_df length:  285


,is_ncbi_bibliography_url,name,parsed_search_term,person_id,pubmed_url,relevant_pubmed_ids,relevant_pubmed_urls,resolved_pubmed_url,search_term
168,False,Lu Chen,"{'exclude': None, 'include': {'author': ['Chen...",90633,https://www.ncbi.nlm.nih.gov/pubmed/?term=Chen...,"[30782829, 30355624, 30068571, 29760176, 25843...","[https://pubmed.ncbi.nlm.nih.gov/30782829, htt...",https://www.ncbi.nlm.nih.gov/pubmed/?term=Chen...,Chen L[Author]
1681,False,Andrew King,"{'exclude': None, 'include': {'author': ['King...",14601,http://www.ncbi.nlm.nih.gov/pubmed?term=King%2...,"[31949136, 31300665, 29136122, 31802997, 28891...","[https://www.ncbi.nlm.nih.gov/pubmed/31949136,...",https://www.ncbi.nlm.nih.gov/pubmed?term=King%...,King A J[au]
4702,False,Yibing Shan,"{'exclude': None, 'include': {'author': ['Shan...",4398,http://www.ncbi.nlm.nih.gov/pubmed?term=Shan%2...,"[24918548, 23374350, 21545110, 20947758]","[https://www.ncbi.nlm.nih.gov/pubmed/24918548,...",https://www.ncbi.nlm.nih.gov/pubmed?term=Shan%...,Shan Y[au]


In [ ]:
processing_editor_parsed_pubmed_links_df = remaining_editor_parsed_pubmed_links_df
if max_editors:
    processing_editor_parsed_pubmed_links_df = processing_editor_parsed_pubmed_links_df[:max_editors]
len(processing_editor_parsed_pubmed_links_df)

In [ ]:
def get_editor_pubmed_paper_ids(europepmc_api: EuropePMCApi, row) -> List[str]:
    parsed_search_term = row.parsed_search_term
    if not parsed_search_term:
        return None
    author_names = parsed_search_term.get('include', {}).get('author')
    try:
        author_names = parsed_search_term.get('include', {}).get('author')
        return europepmc_api.get_author_pmids(author_names)
    except:  # pylint: disable=bare-except
        LOGGER.error('failed to retrieve pubmed ids for author names: %s', author_names, exc_info=1)
        return None


editor_pubmed_links_result_df = processing_editor_parsed_pubmed_links_df[:max_editors].copy()

with europepmc_requests_retry_session() as session:
    europepmc_api = EuropePMCApi(
        session,
        params={'email': email}
    )
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        editor_pubmed_links_result_df['pubmed_ids'] = list(tqdm(
            executor.map(
                lambda row: get_editor_pubmed_paper_ids(europepmc_api, row),
                editor_pubmed_links_result_df.itertuples()
            ),
            total=len(editor_pubmed_links_result_df),
            leave=False
        ))

In [ ]:
non_empty_editor_pubmed_links_result_df = editor_pubmed_links_result_df[
    ~pd.isnull(editor_pubmed_links_result_df['pubmed_ids'])
].copy()
len(non_empty_editor_pubmed_links_result_df)
non_empty_editor_pubmed_links_result_df.head(3)

In [ ]:
non_empty_editor_pubmed_links_result_df['provenance'] = [{
    'source': 'europepmc',
    'imported_timestamp': datetime.utcnow().isoformat()
}] * len(non_empty_editor_pubmed_links_result_df)

In [ ]:
if len(non_empty_editor_pubmed_links_result_df) == 0:
    print('no data to upload')
else:
    print('writing to:', editor_parsed_pubmed_ids_table_name)
    to_gbq(
        non_empty_editor_pubmed_links_result_df,
        project_id=project_id,
        destination_table=editor_parsed_pubmed_ids_table_name,
        if_exists='append'
    )
    print('done')